In [57]:
import sys
import os
import time
import csv
import numpy as np
import pandas as pd
from pyhive import hive
import pickle

In [58]:
os.chdir("/home/hattri10/")

##load the necessary from disk

############################### changed 1_28_2019 #######################

#1.Trained Model


filename='model_unlisted_inscope.sav'

#########################################################################


loaded_model = pickle.load(open(filename, 'rb'))

#2.File Required
MCR_Inscope_All_use=pd.read_pickle("./MCR_Inscope_All_use.pkl")


In [95]:
#### For transforming variables
name1=['NEOPLASMS', 'NERVOUS', 'MUSCULO', 'INJURY', 'DIGESTIVE']
name2=['TX', 'OK', 'IL', 'CA', 'AZ', 'WI', 'OH', 'DE']
name3=['MISC DIAG THERAPEUTIC','OPER DIGESTIVE SYS']
name4=['SPECIALIST','PHARMACY/MEDICAL SUPPLY HOUSE']
name5=['FAMILY PRACTICE/CLINIC','FAMILY PRACTICE','SURGEON','ORTHOPEDIST','OB/GYN','OTOLARYNGOLOGIST','ANESTHESIOLOGIST','OH','MD']
name6=['95999','81599','E1399','84999','64999','29999','99199','49999','76496','92700','A0999','43999','46999','27599']
name7=['821904699', '330685754', '812462824', '263739826', '770701774', '821310149']

################ changed 1_28_2019  ##################
 
dummy_imp=['prov_hist_same_06_s',
 'proc_cd_gal_bin_27599',
 'proc_cd_gal_bin_29999',
 'proc_cd_gal_bin_43999',
 'proc_cd_gal_bin_46999',
 'proc_cd_gal_bin_81599',
 'proc_cd_gal_bin_92700',
 'proc_cd_gal_bin_95999',
 'proc_cd_gal_bin_A0999',
 'proc_cd_gal_bin_E1399',
 'proc_cd_gal_bin_others',
 'prov_same_cpt_den_rate_bin_NoHistory_3',
 'prov_same_cpt_den_rate_bin_[0-0.125]',
 'prov_same_cpt_den_rate_bin_[0.125-1]',
 'st_abbr_cd_bin_AZ',
 'st_abbr_cd_bin_DE',
 'st_abbr_cd_bin_OK',
 'diag_cat_bin_INJURY',
 'diag_cat_bin_MUSCULO',
 'service_billed_dur_bin_(-1.0, 90.0]',
 'den_flag_mcr']
######################################################

bin1=[-1.0, 90, 180, 1056.0]
bin2=[0.0, 40, 60, 91.0]
bin3=[0.0, 5, 37.0]

In [62]:

### read file for daily scoring from hive
### Test out of sample
conn = hive.connect(host='dbslp0500', port= 10002, auth='LDAP', username="hattri10", password="Him<8349")
stat = """set mapreduce.job.queuename= araadh_q1.ararest_sq1"""
stat1="""select * from mcr_eni_redev.test_Apr_may_2019"""
stat1=stat1.replace("\n"," ")
stat1=stat1.replace('\t'," ")
stat1=stat1.replace("   "," ")
cur = conn.cursor()
start=time.time()
print(start)
cur = conn.cursor()
cur.execute(stat)
cur.execute(stat1)
MCR_Inscope= pd.DataFrame(cur.fetchall())
MCR_Inscope.columns = ['key','clm_10','fst_srvc_dt','proc_cd_gal','npi','prov_tin','src_chrg_amt','bth_dt','st_abbr_cd','srvc_prov_catgy_cd','bil_recv_dt','proc_cat','proc_sub_cat','den_flag_mcr','prop_billed_exceed_med','cpt_variation_tagging','service_billed_dur','mem_age','diag_cat','prov_hist_same_03','prov_hist_same_03_prop','prov_hist_same_06','prov_hist_same_06_prop','prov_hist_same_09','prov_hist_same_09_prop','catgy_rol_up_1_desc','catgy_rol_up_2_desc','catgy_rol_up_3_desc','catgy_rol_up_4_desc','prov_catgy_desc','prov_spcl_catgy_desc','ahrq_proc_genl_catgy_desc','prov_spcl_catgy_cd']
print("done")

1562585352.1086416
done


In [63]:
print(MCR_Inscope.shape)


(7827, 33)


In [64]:
MCR_Inscope_oos=MCR_Inscope.copy()
#MCR_Inscope_oos.to_csv("/mapr/projects/MCR_Threshold/Data/MCR_scoring_file_10_09-19_2018.csv",index=False)

In [65]:
MCR_Inscope_oos.shape

(7827, 33)

In [66]:
print(MCR_Inscope_All_use.shape)


(53604, 31)


In [67]:
## Concatenating train & test
MCR_Inscope_All_use['check']='Development'
MCR_Inscope_oos['check']='Out_of_sample'

col=['check','clm_10','fst_srvc_dt','proc_cd_gal','prov_tin','src_chrg_amt','prov_spcl_catgy_desc','st_abbr_cd','srvc_prov_catgy_cd','bil_recv_dt','proc_cat','proc_sub_cat','den_flag_mcr','prop_billed_exceed_med','cpt_variation_tagging','service_billed_dur','mem_age','diag_cat','prov_hist_same_06','prov_hist_same_06_prop']
MCR_Inscope_All_use=MCR_Inscope_All_use[col]
MCR_Inscope_oos=MCR_Inscope_oos[col]


frames = [MCR_Inscope_All_use, MCR_Inscope_oos]
Final = pd.concat(frames)


MCR_Inscope_All=Final.drop_duplicates().reset_index(drop=True)




In [68]:
print(MCR_Inscope_All.shape)

(59535, 20)


In [69]:
MCR_Inscope_All.drop_duplicates(inplace=True)
#MCR_Inscope_All.drop(['npi','bth_dt','812462824_tin_flag'], axis=1, inplace=True)
MCR_Inscope_All.reset_index()
print (MCR_Inscope.shape,MCR_Inscope_All.shape)

MCR_Inscope_All.head(2)

(7827, 33) (59535, 20)


,check,clm_10,fst_srvc_dt,proc_cd_gal,prov_tin,src_chrg_amt,prov_spcl_catgy_desc,st_abbr_cd,srvc_prov_catgy_cd,bil_recv_dt,proc_cat,proc_sub_cat,den_flag_mcr,prop_billed_exceed_med,cpt_variation_tagging,service_billed_dur,mem_age,diag_cat,prov_hist_same_06,prov_hist_same_06_prop
0,Development,3328641134,2017-12-28 00:00:00,55899,620858347,4000.0,UROLOGIST,ME,0996,2018-01-02 00:00:00,Unlisted,surgical,0,1.000000,1,5,64.0,NEOPLASMS,0,0.0
1,Development,3328643764,2017-12-22 00:00:00,69799,391127163,4326.0,OTOLARYNGOLOGIST,ME,0110,2018-01-02 00:00:00,Unlisted,surgical,0,0.909091,1,11,34.0,NERVOUS,0,0.0


In [70]:
# strip (whitespaces) all object dtypes
df_obj = MCR_Inscope_All.select_dtypes(['object'])
df_obj
MCR_Inscope_All[df_obj.columns] = df_obj.apply(lambda x: x.str.strip()).replace('', np.nan)
del [[df_obj]];df_obj = pd.DataFrame();
print(MCR_Inscope_All.shape)

(59535, 20)


In [71]:
#Dates {infer_datetime_format=True}
#MCR_Inscope_All['fst_srvc_dt']=MCR_Inscope_All['fst_srvc_dt'].astype(str).str.strip()
#MCR_Inscope_All['fst_srvc_dt']=pd.to_datetime(MCR_Inscope_All['fst_srvc_dt'], format = "%Y-%m-%d")
#MCR_Inscope_All['lst_srvc_dt']=pd.to_datetime(MCR_Inscope_All['lst_srvc_dt'], format = "%Y-%m-%d")
#MCR_Inscope_All['bil_recv_dt']=pd.to_datetime(MCR_Inscope_All['bil_recv_dt'], format = "%Y-%m-%d")
MCR_Inscope_All['cpt_variation_tagging']=MCR_Inscope_All['cpt_variation_tagging'].astype(str)
#MCR_Inscope_All['den_flag_mcr']=MCR_Inscope_All['den_flag_mcr'].astype(str)

In [72]:
#missing value imputation (Mode)
cols_cat = ['st_abbr_cd','prov_spcl_catgy_desc','diag_cat']
MCR_Inscope_All[cols_cat]=MCR_Inscope_All[cols_cat].fillna(MCR_Inscope_All_use.mode().iloc[0])
#MCR_Inscope_All['prov_gndr']=MCR_Inscope_All['prov_gndr'].replace("U",MCR_Inscope_All_use['prov_gndr'].mode().iloc[0])
print(MCR_Inscope_All.shape)

(59535, 20)


In [73]:
#missing value imputation (median)
#df.apply(lambda x: x.fillna(x.mean()),axis=0)
cols_conti = ['service_billed_dur','mem_age','prop_billed_exceed_med']
MCR_Inscope_All[cols_conti]=MCR_Inscope_All[cols_conti].fillna((MCR_Inscope_All_use[cols_conti].median()))
print(MCR_Inscope_All.shape)

(59535, 20)


In [74]:
#Outlier Treatment
MCR_Inscope_All["mem_age"] = np.where(MCR_Inscope_All["mem_age"] < 0,
                                      MCR_Inscope_All_use['mem_age'].median(),
                                      np.where(MCR_Inscope_All['mem_age'] == 0,
                                               1,
                                               np.where(MCR_Inscope_All['mem_age']> MCR_Inscope_All_use['mem_age'].max(),
                                                        MCR_Inscope_All_use['mem_age'].max(),
                                                        MCR_Inscope_All['mem_age'])))


MCR_Inscope_All['service_billed_dur'] = np.where(MCR_Inscope_All['service_billed_dur'] < 0,
                                                 MCR_Inscope_All_use['service_billed_dur'].median(),
                                                 np.where(MCR_Inscope_All['service_billed_dur']>MCR_Inscope_All_use['service_billed_dur'].max(),
                                                          MCR_Inscope_All_use['service_billed_dur'].max(),
                                                          MCR_Inscope_All['service_billed_dur']
                                                         ))###bucketing for service billed duration 90-180 , above 180 
#q99=MCR_Inscope_All_use['provider_tenure'].quantile(0.995)
#MCR_Inscope_All['provider_tenure'] = np.where(MCR_Inscope_All['provider_tenure'] < 0,MCR_Inscope_All_use['provider_tenure'].median(),np.where(MCR_Inscope_All['provider_tenure'] > q99,q99,MCR_Inscope_All['provider_tenure']))

#q90_03=MCR_Inscope_All_use['prov_hist_same_03'].quantile(0.90)
#MCR_Inscope_All['prov_hist_same_03'] = np.where(MCR_Inscope_All['prov_hist_same_03'] > q90_03, q90_03, MCR_Inscope_All['prov_hist_same_03'])

q90_06=MCR_Inscope_All_use['prov_hist_same_06'].quantile(0.90)
MCR_Inscope_All['prov_hist_same_06'] = np.where(MCR_Inscope_All['prov_hist_same_06'] > q90_06, q90_06, MCR_Inscope_All['prov_hist_same_06'])

#q90_09=MCR_Inscope_All_use['prov_hist_same_09'].quantile(0.90)
#MCR_Inscope_All['prov_hist_same_09'] = np.where(MCR_Inscope_All['prov_hist_same_09'] > q90_09, q90_09, MCR_Inscope_All['prov_hist_same_09'])                                        
print(MCR_Inscope_All.shape)

(59535, 20)


In [75]:
# Variable Bucketization
# a.Categorical variables

## 1.Diagnosis code category


MCR_Inscope_All['diag_cat_bin'] = MCR_Inscope_All.diag_cat.where(MCR_Inscope_All.diag_cat.isin(name1), 'others')
#MCR_Inscope_All['diag_cat_bin'].unique()

## 2.Provider State category

MCR_Inscope_All['st_abbr_cd_bin'] = MCR_Inscope_All.st_abbr_cd.where(MCR_Inscope_All.st_abbr_cd.isin(name2), 'others')
#MCR_Inscope_All['st_abbr_cd_bin'].unique()

##3.CPT high level description

#MCR_Inscope_All['ahrq_proc_genl_catgy_desc_bin'] = MCR_Inscope_All.ahrq_proc_genl_catgy_desc.where(MCR_Inscope_All.ahrq_proc_genl_catgy_desc.isin(name3), 'others')
#MCR_Inscope_All['ahrq_proc_genl_catgy_desc_bin'].unique()

##4.catgy_rol_up_2_desc servicing provider catgy code high level description

#MCR_Inscope_All['catgy_rol_up_2_desc_bin'] = MCR_Inscope_All.catgy_rol_up_2_desc.where(MCR_Inscope_All.catgy_rol_up_2_desc.isin(name4), 'others')
#MCR_Inscope_All['catgy_rol_up_2_desc_bin'].unique()

##5.Speciality code high level description

MCR_Inscope_All['prov_spcl_catgy_desc_bin'] = MCR_Inscope_All.prov_spcl_catgy_desc.where(MCR_Inscope_All.prov_spcl_catgy_desc.isin(name5), 'others')
#MCR_Inscope_All['prov_spcl_catgy_desc_bin'].unique()

##6 tin flag
MCR_Inscope_All['tin_flag'] = MCR_Inscope_All.prov_tin.where(MCR_Inscope_All.prov_tin.isin(name7), 'others')

##7 CPT grouping
MCR_Inscope_All['proc_cd_gal_bin'] = MCR_Inscope_All.proc_cd_gal.where(MCR_Inscope_All.proc_cd_gal.isin(name6), 'others')
print(MCR_Inscope_All['proc_cd_gal_bin'].unique())

['others' '29999' '64999' 'E1399' '27599' '81599' '43999' '95999' '46999'
 '84999' 'A0999' '99199' '49999' '76496' '92700']


In [76]:
MCR_Inscope_All.shape

(59535, 25)

In [77]:
# b. Continous Variable

## 1.Service Billed Duration


MCR_Inscope_All['service_billed_dur_bin'] = pd.cut(MCR_Inscope_All['service_billed_dur'], bin1)

## 2. Member Age


MCR_Inscope_All['mem_age_bin'] = pd.cut(MCR_Inscope_All['mem_age'], bin2)


## 3. Provider Tenure

#bin3 = [MCR_Inscope_All['provider_tenure'].min()-1,5,MCR_Inscope_All['provider_tenure'].max()+1]
#MCR_Inscope_All['provider_tenure_bin'] = pd.cut(MCR_Inscope_All['provider_tenure'], bin3)



In [78]:
MCR_Inscope_All.shape

(59535, 27)

In [79]:
MCR_Inscope_All['prov_hist_same_06_s']=(MCR_Inscope_All['prov_hist_same_06']-MCR_Inscope_All_use['prov_hist_same_06'].min())/(MCR_Inscope_All_use['prov_hist_same_06'].max()-MCR_Inscope_All_use['prov_hist_same_06'].min())


MCR_Inscope_All['prov_same_cpt_den_rate_bin']=np.where(MCR_Inscope_All['prov_hist_same_06_prop'].isnull(), "NoHistory_3",
                                             np.where(MCR_Inscope_All['prov_hist_same_06_prop']<0.125,"[0-0.125]",
                                                      np.where(MCR_Inscope_All['prov_hist_same_06_prop']>=0.125,"[0.125-1]","Other")))
MCR_Inscope_All['prov_same_cpt_den_rate_bin'].unique()

array(['[0-0.125]', 'NoHistory_3', '[0.125-1]'], dtype=object)

In [80]:
MCR_Inscope_All.shape

(59535, 29)

In [81]:
################ changed 1_28_2019  ##################

dev=MCR_Inscope_All[['proc_cd_gal_bin','prov_same_cpt_den_rate_bin','st_abbr_cd_bin','den_flag_mcr','prov_hist_same_06_s','diag_cat_bin','service_billed_dur_bin']]

######################################################

In [82]:
dev.shape

(59535, 7)

In [86]:
len(dummy_imp)

21

In [88]:

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix , classification_report, accuracy_score, roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score


/app_2/jupyter/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [96]:

df_d=pd.get_dummies(dev)
df_d=df_d[dummy_imp]
X=df_d.drop('den_flag_mcr',axis=1)

print( X.shape)
y=df_d['den_flag_mcr'].values
print (y.shape)



### Separating out of sample
y_pred = loaded_model.predict(X)
y_prob=loaded_model.predict_proba(X)[:,1]
MCR_Inscope_All['pred']=y_pred
MCR_Inscope_All['pred_proba']=y_prob
data_oos=MCR_Inscope_All[MCR_Inscope_All['check']=='Out_of_sample']

##out of sample results

print(classification_report(data_oos['den_flag_mcr'],data_oos['pred']))
print(confusion_matrix(data_oos['den_flag_mcr'],data_oos['pred']))

type(data_oos)

(59535, 20)
(59535,)
              precision    recall  f1-score   support

           0       0.90      0.76      0.82      5487
           1       0.56      0.78      0.65      2152

   micro avg       0.76      0.76      0.76      7639
   macro avg       0.73      0.77      0.74      7639
weighted avg       0.80      0.76      0.77      7639

[[4159 1328]
 [ 478 1674]]


pandas.core.frame.DataFrame

In [97]:
## applying threshold

## Applying threshold

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix , classification_report, accuracy_score, roc_curve, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, cross_val_score

################# changed 1_28_2019 ####################

threshold=0.3

########################################################

c=[]

for i in data_oos['pred_proba']:
    
    if i > threshold:
        d=[1]
        c.extend(d)
      
    else:
        d=[0]
        c.extend(d)
        
data_oos['pred1']=c


# Print confusion Matrix
from sklearn.metrics import confusion_matrix

print(classification_report(data_oos['den_flag_mcr'], data_oos['pred1']))
print(confusion_matrix(data_oos['den_flag_mcr'], data_oos['pred1']))


              precision    recall  f1-score   support

           0       0.94      0.44      0.59      5487
           1       0.39      0.93      0.55      2152

   micro avg       0.57      0.57      0.57      7639
   macro avg       0.67      0.68      0.57      7639
weighted avg       0.79      0.57      0.58      7639

[[2387 3100]
 [ 154 1998]]


/app_2/jupyter/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [39]:
#data_oos.head()
data_oos.to_csv("/mapr/projects/MCR_Threshold/Data/dry_run_others.csv",index=False)

data_clm=data_oos.groupby('clm_10', as_index=False).agg({"pred1": "max"})
c=[]
for i in data_clm['pred1']:
    if i ==1:
        d=['High']
        c.extend(d)
    else:
        d=['Low']
        c.extend(d)    

data_clm['status']=c

data_new_sc=data_clm.copy()
data_new_sc['icn']=data_new_sc['clm_10']
data_new_sc['score']=data_new_sc['status']
data_new_sc['date']= pd.to_datetime(pd.datetime.now().date(), format = "%Y-%m-%d") 
data_new_sc=data_new_sc.drop(['clm_10','pred1','status'], axis=1)
print data_new_sc.head()
data_new_sc.to_csv("/mapr/projects/MCR_Threshold/Data/dry_run_others_claim_status.csv",index=False)


          icn score       date
0  7615493110   Low 2019-02-22
1  7619922078  High 2019-02-22
2  7623685186  High 2019-02-22
3  7623818422  High 2019-02-22
4  7623819390  High 2019-02-22


In [40]:
### Need to ingest data_new_sc (newly scored outcome) in hive directly under the name eni_test.newly_scored